# Initial value problems
---

This notebook shows how to use explicit single-step methods to solve initial value problems (IVP). We start with a simple implementation of the forward Euler's method before we show how to use an ODE solver from the SciPy module *integrate*.

## Explicit single-step methods

We consider the differential equation
$$
\dot{\textbf{x}}(t) = \textbf{f}(t, \textbf{x}; \Theta), \quad \textbf{x}(t_0) = \textbf{x}_0
$$

- We start at $t_0$ with the initial value $\textbf{x}_0$
- Step by step we calculate further points $\textbf{x}_k\approx\textbf{x}(t_k)$ at discrete time steps $t_k = t_0 + k h$
$$
\textbf{x}_{k+1} = \textbf{x}_{k} + h F(t, \textbf{x}_{k};\Theta), \quad k=1,2,\dots
$$

## Euler's method

- From the initial value $x(t_0)$ take a step of size $h$ along the tangent, i.e. derivative at $t_0$
- This gives us the update formula
$$
x_{k+1} = x_{k} + h f(t_k, x_k)
$$

<img src="figures/ODE_Euler_single_step.png" alt="One step of the explicit Euler's method" width="400"/>

In [ ]:
import numpy as np
def Eulers_method(dxdt, x0, t0, tmax, N):
    '''Simple implementation of the forward Euler's method
    
    This implementation takes the initial and final time as well as the number of time steps.
    It calculate a uniform step size and returns a numpy array with the solution at each time step.
    
    Parameters
    ----------
    dxdt : function
        Derivative function
    x0 : float
        Initial value
    t0 : float
        Initial time
    tmax : float
        Final time
    N : int
        Number of time steps
    
    Returns
    -------
    t : numpy array
        Time steps
    x : numpy array
        Solution at the time steps    
    '''
    t = np.linspace(t0, tmax, N)
    x = x0 * np.ones(N)

    for i in range(1, N):
        x[i] = x[i-1] + (t[i] - t[i-1]) * dxdt(t[i-1], x[i-1])
    
    return t, x

### Exponential growth example

We consider exponential growth with an inhibiting term which can be modelled by
$$
x'(t) = 4 \exp(0.8t) - 0.5 x = f(t, x)
$$
from $t=0$ to $t=4$ and with $x(t=0) = 2$

The true solution can be found analytically as
$$
x(t) = \frac{4}{1.3}\left(\exp(0.8t) - \exp(-0.5t)\right) + 2\exp(-0.5t)
$$

**Remark:**
We are using *lambda* functions but you could also implement it with the standard function definition *def*

In [ ]:
tmin = 0
tmax = 4
# Exact solution
xexact = lambda t: 4/1.3 * (np.exp(0.8*t) - np.exp(-0.5*t)) + 2*np.exp(-0.5*t)

# Update function
dx = lambda t, x : (4*np.exp(0.8*t) - 0.5 * x)

t, x = Eulers_method(dx, 2, tmin, tmax, 9)

import matplotlib.pyplot as plt
plt.plot(t, x, 'k-o')

tt = np.linspace(tmin, tmax, 100)
plt.plot(tt, xexact(tt), 'b')
plt.show()

### Bacterial population density dynamics

- Consider the bacterial population density $x(t)$ which has the growth rate $\alpha - \beta x$
- This leads to the first order differential equation
$$
\dot{x}(t) = (\alpha - \beta x(t)) x(t)
$$
where $\alpha>0$ with units s$^{-1}$ and $\beta>0$ with units m$^2\,$s$^{-1}$
- Evaluate the system for the initial condition $x(0) = 1$ and with the parameters $\alpha=2$ and $\beta=0.2$

#### Part a) 

Calculate a few steps of the forward Euler's method for this problem with step size $\Delta t = 1$.

Define the function and apply Euler's method defined above.

In [ ]:
tmin = 0
tmax = 10
N = 11
alpha = 2
beta = 0.2

# Update function
dx = lambda t, x: (alpha - beta*x) * x

t, x = Eulers_method(dx, 1, tmin, tmax, N)

import matplotlib.pyplot as plt
plt.plot(t, x, 'k-o')
plt.xlabel("Time")
plt.ylabel("x(t)")
plt.title("Solution for a time step of {}".format((tmax-tmin)/(N-1)))
plt.show()

We can see that the solution oscillates around the value of 10 and that the amplitude of the oscillations seems to decrease.

#### Part b) 

Apply Euler's method with a time steps of 0.25, 0.5 and 1.25. Explain the results.

In [ ]:
Nlist = [9, 21, 41]

for N in Nlist:
    t, x = Eulers_method(dx, 1, tmin, tmax, N)

    import matplotlib.pyplot as plt
    plt.plot(t, x, 'k-o')
    plt.xlabel("Time")
    plt.ylabel("x(t)")
    plt.title("Solution for a time step of {}".format((tmax-tmin)/(N-1)))
    plt.show()

The method converges to the correct solution for $h\le1$ but shows oscillations for values of $0.5<h\le1$. For values of $h>1$ the method diverges.


## Python ODE solvers

### Stiff problem from the lecture slides

In [ ]:
def stiff_ODE(t, x, k1, k2, cm):
# Stiff reaction ODE system
    dxdt = np.zeros(2)
    dxdt[0] = -k1 * cm * x[0]
    dxdt[1] = k1 * cm * x[0] - k2 * x[1]

    return dxdt

Solve the stiff ODE system with different ODE solver from `scipy.integrate`. Test the standard explicit Runge-Kutta method of order 5(4) and the implicit Runge-Kutta method of the Radau IIA family of order 5.

In [ ]:
# ODE_stiff_solve.m - solve stiff ODE system with different ODE solvers
# Compare ode45 and ode15s for systems with increasing stiffness
import numpy as np
from scipy.integrate import solve_ivp
import time

k1 = 1
k2 = 1
cm = 1
# Initial values
c0 = [1, 0]

time_Radau = []
time_RK45 = []
k2s = []
sols = []
# Solve the ODE with different solvers and reaction rates
for i in range(6):
    k2s.append(k2)
    start = time.time()
    print("Start time for k2={} is {:0.6f} seconds".format(k2, start))
    sol = solve_ivp(stiff_ODE, [0, 10], c0, method='Radau', rtol=1e-3, args=(k1, k2, cm))
    time_Radau.append(time.time()-start)
    
    # Add the solution with the Radau method to the list
    sols.append(sol)
    
    start = time.time()
    sol = solve_ivp(stiff_ODE, [0, 10], c0, method='RK45', rtol=1e-3, args=(k1, k2, cm))
    time_RK45.append(time.time()-start)
    
    k2 = k2 * 10

In [ ]:
import matplotlib.pyplot as plt
plt.semilogy(np.log(k2s), time_Radau, 'rx--', label="Radau")
plt.semilogy(np.log(k2s), time_RK45, 'g*--', label="RK45")
plt.xlabel("log(k2s)")
plt.ylabel("Solution time [s]")
plt.legend()
plt.show()

In [ ]:
# Generate the output for the LaTeX table for the lecture slides
sRK45 = str()
sRadau = str()
for i, j in zip(time_RK45, time_Radau):
    sRK45 = sRK45 + "{:4.4f} & ".format(i)
    sRadau = sRadau + "{:4.4f} & ".format(j)

print("RK45:", sRK45)
print("Radau:", sRadau)

#### Plot the solution for different values of k$_2$

In [ ]:
import matplotlib.pyplot as plt

# Define the plot
fig, ax = plt.subplots(1, 1, figsize=(15, 6))

i = 2
sol = sols[i]
ax.plot(sol.t, sol.y[0], 'k-', label="A")
ax.plot(sol.t, sol.y[1], 'r-.', label="A*")
ax.plot(sol.t, 1 - sol.y[0] - sol.y[1], 'g:', label="B")
ax.set_xlabel("Nondimensional time [-]", fontsize=14)
ax.set_ylabel("Nondimensional concentration [-]", fontsize=14)
ax.set_title(r"Concentrations in the reactor for k$_2$={}".format(k2s[i]))
ax.set_yscale('log')
plt.xlim([0.00001, np.max(sol.t)])
ax.legend()
plt.show()